# Imports

In [1]:
import sklearn.utils.class_weight as wgt
import numpy as np
import tensorflow as tf
import os
import random
import shutil as shtl
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import sys
sys.path.append('../../')
import utils
tfk = tf.keras 
tfkl = tfk.layers

Init Plugin
Init Graph Optimizer
Init Kernel


# Setting Seed

In [2]:
seed = 10

np.random.seed(seed)
tf.random.set_seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
tf.compat.v1.set_random_seed(seed)

# Building Model

In [4]:
training_dir = os.path.join('.', 'training')
validation_dir = os.path.join('.', 'validation')
unsampled_data_dir = os.path.join('.', 'data')
min_images = 264

def downsampling(path_to_original, unsampled_data_dir):
    os.mkdir(unsampled_data_dir)
    for _dir in os.listdir(path_to_original):
        class_dir_original = os.path.join(path_to_original, _dir)
        if (os.path.isdir(class_dir_original)):
            class_unsampled_dir = os.path.join(unsampled_data_dir, _dir)
            os.mkdir(class_unsampled_dir)
            files = []
            for f in os.listdir(class_dir_original): 
                files.append(f)
            random.shuffle(files)
            files = files[:min_images]
            for d in files:
                shtl.copyfile(os.path.join(class_dir_original, d), os.path.join(class_unsampled_dir, d))
            
            
downsampling('../../data', unsampled_data_dir)
utils.build_train_valid_dirs(unsampled_data_dir, training_dir, validation_dir)
            

batch_size = 64
epochs = 1000
patience = 100
input_shape = (128, 128, 3)
neurons = [512, 256]
hiddens = 2

In [5]:
train_data_gen = ImageDataGenerator(rescale=1/255.,
                                    rotation_range=45,
                                    zoom_range=[0.,0.2],
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    height_shift_range=0.3,
                                    width_shift_range=0.3)

valid_data_gen = ImageDataGenerator(rescale=1/255.)

train_gen = train_data_gen.flow_from_directory(directory=training_dir,
                                               target_size=input_shape[:2],
                                               interpolation='bilinear',
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               classes=None,
                                               shuffle=True,
                                               seed=seed)  

valid_gen = valid_data_gen.flow_from_directory(directory=validation_dir,
                                               target_size=input_shape[:2],
                                               interpolation='bilinear',
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               classes=None,
                                               shuffle=False,
                                               seed=seed)  
                                        

Found 3136 images belonging to 14 classes.
Found 560 images belonging to 14 classes.


In [6]:
tl_model = utils.build_tl_vgg_model(hiddens=hiddens, neurons=neurons, input_shape=input_shape, seed=seed)

# Compile the model
tl_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics=['accuracy','categorical_accuracy', 'Precision', 'Recall'])
tl_model.summary()

Metal device set to: Apple M1 Pro


2021-11-28 17:58:16.854547: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-11-28 17:58:16.854667: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
Flattening (Flatten)         (None, 8192)              0         
_________________________________________________________________
dropout (Dropout)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               4194816   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328

In [ ]:
tl_history = tl_model.fit(
    x = train_gen,
    batch_size = batch_size,
    epochs = 1,
    validation_data = valid_gen,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=patience, restore_best_weights=True)]
).history

# Training and Results Visualization

In [ ]:
name = utils.get_name_model(hiddens=hiddens, neurons=neurons)
tl_model.save(name)